## Experiment: point cloud registration for evaluation 
Motivation: The reconstructed point clouds has no direction transformation to annotated mesh file. This notebook tries to 
1. Use point cloud registration method to first predict a transformation from rtabmap point clouds to annotated point clouds
2. Convert HAIS segmentation result to bounding boxes 
3. Eval the detection result on matterport3D/rtabmap 
4. Compare the results to check the registration performance   

In [1]:
import os 
import sys 
import numpy as np 
import open3d as o3d 
import matplotlib.pyplot as plt 
import plotly
import pickle 
%cd /home/junting/habitat_ws/src/struct-nav/habitat_recognition/scripts

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
/home/junting/habitat_ws/src/struct-nav/habitat_recognition/scripts


### Matterport3D 
Evaluate prediction result on matterport3d original point cloud 

In [2]:
mp3d_scans_dir = "/media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans"
scene = '17DRP5sb8fy' # middle-sized common flat 
scene_dir = os.path.join(mp3d_scans_dir, scene)

In [3]:
############ label mapping ###################
import csv 
mapping_mpcat40id_to_nyu40id = {}

with open("./metadata/mpcat40_to_nyu40.tsv") as csvfile:
    reader = csv.DictReader(csvfile, delimiter="\t")
    for row in reader:
        mapping_mpcat40id_to_nyu40id[int(row['mpcat40index'])] = int(row['nyu40id'])

mapping_nyu40id_to_nyu40class = {}
with open("./metadata/mp3d_mapping.tsv") as csvfile:
    reader = csv.DictReader(csvfile, delimiter="\t")
    for row in reader:
        try:
            mapping_nyu40id_to_nyu40class[int(row['nyu40id'])] = row['nyu40class']
        except:
            print("Error:", row['nyu40id'])

Error: 


In [4]:
# run model inference 
from models.segmenter_hais import SegmenterHAIS
from config.config_hais import ConfigHAIS

config = ConfigHAIS()
seg_hais = SegmenterHAIS(config)

[2022-02-10 10:43:53,601  INFO  log.py  line 39  103813]  ************************ Start Logging ************************
INFO - 2022-02-10 10:43:53,601 - log - ************************ Start Logging ************************


cuda available: True


[2022-02-10 10:43:55,161  INFO  utils.py  line 67  103813]  Restore from /home/junting/project_cvl/HAIS/pretrain/hais_ckpt.pth
INFO - 2022-02-10 10:43:55,161 - utils - Restore from /home/junting/project_cvl/HAIS/pretrain/hais_ckpt.pth


In [6]:
ply_path = os.path.join(scene_dir, f"{scene}_semantic.ply")
o3d_pcl = o3d.io.read_point_cloud(ply_path)
down_sampled_o3d_pcl, pred_result = seg_hais.predict(o3d_pcl)

Original point cloud has 924711 points, subsampled to 300000 points by random method


In [34]:
print(pred_result.keys())
print(pred_result['inst_mask'].shape)

dict_keys(['inst_mask', 'inst_conf', 'label_id'])
(56, 300000)


In [35]:
########## convert segmentation to batch_pred_map_cls ###########
from utils.box_util import get_3d_box

pred_map_cls = []
points = np.asarray(down_sampled_o3d_pcl.points)

for inst_id, inst_label in enumerate(pred_result['label_id']):
    inst_points = points[pred_result['inst_mask'][inst_id], :]
    inst_min = inst_points.min(0)
    inst_max = inst_points.max(0)
    center = (inst_min + inst_max) / 2.0
    size = inst_max - inst_min
    aabb = get_3d_box(size, 0, center)
    
    # (pred_cls, pred_box_params, score)
    pred_map_cls.append((
        pred_result['label_id'][inst_id],
        aabb,
        pred_result['inst_conf'][inst_id]
    ))

batch_pred_map_cls = [pred_map_cls]

In [44]:
########## convert gt annotations to batch_gt_map_cls ############
from dataset.mp3d.utils import read_house_file, getOBB

gt_map_cls = []

house_file = os.path.join(scene_dir, f"{scene}.house")
house_dict, parse_dict = read_house_file(house_file)
# C category_index category_mapping_index category_mapping_name mpcat40_index mpcat40_name 0 0 0 0 0
mapping_rawcat_to_mpcat40 = {}
for cat_dict in house_dict['C']:
    mapping_rawcat_to_mpcat40[int(cat_dict['category_index'])] = \
        int(cat_dict['mpcat40_index'])

# O object_index region_index category_index px py pz  a0x a0y a0z  a1x a1y a1z  r0 r1 r2 0 0 0 0 0 0 0 0 
for object_dict in house_dict['O']: 

    raw_label = int(object_dict['category_index'])
    if raw_label == -1:
        nyu40_label = 0
    else:
        mpcat40_label = mapping_rawcat_to_mpcat40[raw_label]
        nyu40_label = mapping_mpcat40id_to_nyu40id[mpcat40_label]
    
    center, boxRotation, radii = getOBB(object_dict)
    o3d_obb = o3d.geometry.OrientedBoundingBox(center, boxRotation, 2*radii)
    o3d_aabb = o3d_obb.get_axis_aligned_bounding_box()
    center = o3d_aabb.get_center()
    size = o3d_aabb.get_extent()
    aabb = get_3d_box(size, 0, center)

    gt_map_cls.append((
        nyu40_label,
        aabb
    ))
batch_gt_map_cls = [gt_map_cls]

In [40]:
house_dict['C'][0]

{'category_index': '0',
 'category_mapping_index': '1',
 'category_mapping_name': 'wall',
 'mpcat40_index': '1',
 'mpcat40_name': 'wall'}

In [47]:
from eval.ap_helper import APCalculator


""" Accumulate one batch of prediction and groundtruth.

Args:
    batch_pred_map_cls: a list of lists [[(pred_cls, pred_box_params, score),...],...]
    batch_gt_map_cls: a list of lists [[(gt_cls, gt_box_params),...],...]
        should have the same length with batch_pred_map_cls (batch_size)
"""
ap_calculator = APCalculator(0.25, mapping_nyu40id_to_nyu40class)
ap_calculator.step(batch_pred_map_cls, batch_gt_map_cls)
metrics_dict = ap_calculator.compute_metrics()
for key in metrics_dict:
    print('eval %s: %f'%(key, metrics_dict[key]))

/home/junting/habitat_ws/src/struct-nav/habitat_recognition/scripts/eval/eval_det.py:159: RuntimeWarning: invalid value encountered in true_divide
  rec = tp / float(npos)
/home/junting/habitat_ws/src/struct-nav/habitat_recognition/scripts/eval/eval_det.py:159: RuntimeWarning: invalid value encountered in true_divide
  rec = tp / float(npos)


28 nan
39 0.0
9 0.0
5 0.0
11 0.0
16 0.0
3 0.0
8 0.07142857142857142
6 0.0
7 0.0
14 nan
4 0.0
36 0.0
34 0.0
40 0
22 0
21 0
1 0
0 0
2 0
17 0
19 0
27 0
33 0
18 0
25 0
15 0
12 0
eval void Average Precision: 0.000000
eval wall Average Precision: 0.000000
eval floor Average Precision: 0.000000
eval cabinet Average Precision: 0.000000
eval bed Average Precision: 0.000000
eval chair Average Precision: 0.000000
eval sofa Average Precision: 0.000000
eval table Average Precision: 0.000000
eval door Average Precision: 0.071429
eval window Average Precision: 0.000000
eval picture Average Precision: 0.000000
eval counter Average Precision: 0.000000
eval desk Average Precision: nan
eval shelves Average Precision: 0.000000
eval curtain Average Precision: 0.000000
eval dresser Average Precision: 0.000000
eval pillow Average Precision: 0.000000
eval mirror Average Precision: 0.000000
eval clothes Average Precision: 0.000000
eval ceiling Average Precision: 0.000000
eval television Average Precision: 0.00